# We start off by reading the data

In [ ]:
import pandas as pd
import plotly.graph_objects as go
BL_KURZEL = ['BB', 'BE', 'BW', 'BY', 'DE', 'HB', 'HE', 'HH', 'MV', 'NI', 'NW', 'RP', 'SH', 'SL', 'SN', 'ST', 'TH']
dfs = {}
for kurzel in BL_KURZEL:
    dfs[kurzel] = pd.read_pickle(f'data/df_vac_{kurzel}.pkl')

pd.options.plotting.backend = "plotly"
# colors
import plotly.express as px
from itertools import cycle
palette = cycle(px.colors.qualitative.Safe)
#palette = cycle(['black', 'grey', 'red', 'blue'])
#palette = cycle(px.colors.sequential.PuBu)


## Let's take a look

In [ ]:
dfs['DE'].tail(5)

In [ ]:
current_official_doses = dfs['DE'].tail(1)['dosen_kumulativ']
#current_official_doses.values[0]
current_official_doses


In [ ]:
dfs['DE'].set_index('date')['dosen_kumulativ'].plot.line(width=1000, height=500, title='Impfungen Deutschland').show()


In [ ]:
to_plot = dfs['DE'].set_index('date')
fig = go.Figure()
def add_dosen_for_bl(kurzel):
    to_plot = dfs[kurzel].set_index('date')
    fig.add_trace(go.Scatter(x = to_plot.index,
                             y=to_plot['dosen_kumulativ'],
                             mode='lines',
                             name=f'{kurzel}',
                             marker_color=next(palette),
                             line=dict( width=4)))

for kurzel in BL_KURZEL:
    add_dosen_for_bl(kurzel)

fig.update_layout(
        width=1200,
        height=600,
        title='Total doses'
    )
fig.show()

### Average doses of the last days

In [ ]:
def avg_doses_of_last_x_days(last_days):
    data = []
    for i in range(1, last_days):
        data.append([i, int(dfs['DE'].tail(i)['vortag_dosen_dif'].sum() /i)])

    a = pd.DataFrame(columns=["Last x days", "Average vacs"], data=data)

    fig = go.Figure(data=[
        go.Bar(name='Three',x=a['Last x days'], y=a['Average vacs'])

    ])
    fig.update_layout(
        width=1000,
        height=350,
        title='Average doses of the last x days'
    )
    fig.show()
    return a


In [ ]:
a = avg_doses_of_last_x_days(100)
a.head(7)

In [ ]:
a = avg_doses_of_last_x_days(365)



In [ ]:


fig7 = go.Figure()
fig21 = go.Figure()

def add_rolling(fig, df, days, kurzel):
    fig.add_trace(go.Scatter(x = df.index,
                         y=df.vortag_dosen_dif.rolling(days).mean(),
                         mode='lines',
                         name=f'{kurzel}',
                         marker_color=next(palette),
                         line=dict( width=4)))

def add_traces(fig, df):
    fig.add_trace(go.Scatter(x = df.index,
                             y=df.vortag_dosen_dif,
                             mode='markers',
                             name='Real',
                             line=dict(color='grey', width=1)))


for kurzel in BL_KURZEL:
    to_plot = dfs[kurzel].set_index('date')

    #add_traces(fig7, to_plot)
    #add_traces(fig21, to_plot)
    add_rolling(fig7, to_plot, 7, kurzel)
    add_rolling(fig21, to_plot, 21, kurzel)



fig7.update_layout(
    width=1000,
    height=500,
    yaxis_range=[0,400_000],
    title='Rolling 7 day window vaccinations'
)

fig21.update_layout(
    width=1000,
    height=500,
    yaxis_range=[0,400_000],
    title='Rolling 21 day window vaccinations'
)
fig7.show()
fig21.show()

## Vaccinations per weekday

In [ ]:

to_plot_all = dfs['DE'].groupby(["weekday_name"])['vortag_dosen_dif'].mean().sort_values()
to_plot_10 = dfs['DE'].tail(7*10).groupby(["weekday_name"])['vortag_dosen_dif'].mean().sort_values()
to_plot_5 = dfs['DE'].tail(7*5).groupby(["weekday_name"])['vortag_dosen_dif'].mean().sort_values()
to_plot_2 = dfs['DE'].tail(7*2).groupby(["weekday_name"])['vortag_dosen_dif'].mean().sort_values()


fig = go.Figure(data=[
    go.Bar(name='complete timeline', y=to_plot_all, x=to_plot_all.index),
    go.Bar(name='10 weeks', y=to_plot_10, x=to_plot_all.index),
    go.Bar(name='5 weeks', y=to_plot_5, x=to_plot_all.index),
    go.Bar(name='2 weeks', y=to_plot_2, x=to_plot_all.index),
])
fig.update_layout(
    width=1000,
    height=500,
    title='Average vaccinations per weekday'
)
fig.show()

## Vaccinations per calendar week

In [ ]:
def vac_per_calendar_week(df, where):
    # tail(length - 5) removes 53rd calendar week
    to_plot_sum = df.tail(len(df.index) - 8).groupby(["calendar_week"])['vortag_dosen_dif'].sum()
    to_plot_mean = df.tail(len(df.index) - 8).groupby(["calendar_week"])['vortag_dosen_dif'].mean()

    fig = go.Figure(data=[
        go.Bar(name='Sum', y=to_plot_sum),
        go.Bar(name='Mean', y=to_plot_mean),
    ])
    fig.update_layout(
        width=1000,
        height=500,
        title=f'Vaccinations per calendar week ({where})'
    )
    fig.show()

vac_per_calendar_week(dfs['DE'], 'Deutschland')
vac_per_calendar_week(dfs['BY'], 'Bayern')

